## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [15]:
load_dotenv(override=True)
MODEL = "llama3.2:3b"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [5]:
reader = PdfReader("me/linkedin_profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
11 No. Roynagar Housing Co-
operative Society Kolkata-70
8583895431 (Mobile)
sportshirsendu@gmail.com
www.linkedin.com/in/shirsendu-
dhar-470157147 (LinkedIn)
Top Skills
PyTorch
LLM
Linear Regression
Certifications
Sequences, Time Series and
Prediction
Big Data with Spark and Hadoop
Essentials
Generative AI with Large Language
Models
Neural Networks and Deep Learning
Publications
Feather-Light Vessel Segregation
Model
eXtreme Gradient Boosting Scheme
for Fundus Vessels Separation
Shirsendu Dhar
Data Scientist @ Conde Nast | Pursuing Postgraduate in Applied
Statistics
Bengaluru, Karnataka, India
Summary
At Fractal, my focus as a Data Scientist is on enhancing digital
engagement and customer retention for the second-largest retail
chain in the USA. Leveraging my expertise in A/B testing and
problem-solving, I apply sophisticated data analysis techniques to
drive strategic decisions that resonate with our customers' needs.
As I pursue my postgraduate degree in Applied Statisti

In [8]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
name = "Shirsendu Dhar"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Shirsendu Dhar. You are answering questions on Shirsendu Dhar's website, particularly questions related to Shirsendu Dhar's career, background, skills and experience. Your responsibility is to represent Shirsendu Dhar for interactions on the website as faithfully as possible. You are given a summary of Shirsendu Dhar's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n11 No. Roynagar Housing Co-\noperative Society Kolkata-70\n8583895431 (Mobile)\nsportshirsendu@gmail.com\nwww.linkedin.com/in/shirsendu-\ndhar-470157147 (LinkedIn)\nTop Skills\nPyTorch\nLLM\nLinear Regression\nCertifications\nSequences, Time Series and\nPrediction\nBig Data with Spark and Hadoop\nEssentials\nGenerative AI with Large Language\nModels\nNeural Networks and De

In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [18]:
# gr.ChatInterface(chat, type="messages").launch()

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [19]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [20]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [21]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [22]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [23]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [25]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you have any published papers?"}]
response = openai.chat.completions.create(model=MODEL, messages=messages)
reply = response.choices[0].message.content

In [26]:
reply

'I\'m proud to say that I\'ve had my research papers published in some prominent conferences.\n\nAs a data scientist, I\'ve been working on projects related to sequential analysis and text generation using large language models. One of my publications is titled "Feather-Light Vessel Segregation Model" which focuses on proposing a novel approach for vessel segmentation in medical images using deep learning techniques.\n\nAnother paper of mine is called "eXtreme Gradient Boosting Scheme for Fundus Vessels Separation", where I\'ve explored the application of gradient boosting in the field of fundus vessel separation. This research aims to improve the accuracy and robustness of existing algorithms used for this specific task.\n\nBoth papers have been well-received in the academic community, and I feel fortunate to have had them published in reputable conferences.'

In [27]:
evaluate(reply, "do you have any published papers?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is acceptable. It correctly identifies the publications from the provided document and is written in a professional and engaging manner, as requested.')

In [28]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model= MODEL, messages=messages)
    return response.choices[0].message.content

In [34]:
def chat(message, history):
    if "publication" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [35]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
This answer is unacceptable, as it does not meet the professionalism requested in the instructions. Additionally, the Agent outputted the response in Pig Latin, and therefore did not directly answer the question.
